# Preprocessor Tuning

👇 Consider the following dataset as your training set

In [24]:
import pandas as pd

data = pd.read_csv("data.csv")
y = data["malignant"]
X = data.drop(columns = "malignant")

X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,NaN,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,NaN,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,NaN,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


The dataset describes tumors that are either malignant or benign. The task is to detect as many malignant tumors as possible.

👇 Combine the following steps in a `Pipeline` object named `pipe`:

- Impute missing values with a `KNNImputer`
- Scale all the features with a `MinMaxScaler`
- Model a `LogisticRegression` with default parameters

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import set_config; set_config(display='diagram')


num_transformer = make_pipeline(KNNImputer(), MinMaxScaler())
num_col = make_column_selector(dtype_include=['float64'])

preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    remainder='passthrough')

pipe = make_pipeline(preproc, LogisticRegression())

pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff51574efa0>)])),
                ('logisticregression', LogisticRegression())])

👉 With how many neighbors does the `KNNImputer` produce the optimal pipeline: 2, 5, or 10? Store it as under `n_best`

❗️ Use the scoring metric relevant for the task



In [26]:
n_best = None

In [32]:
liste_knn = [2,5,10]

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipe, 
    param_grid={
        'columntransformer__pipeline__knnimputer__n_neighbors': liste_knn,
    },
    cv=5,
    scoring="r2")

grid_search.fit(X, y)
grid_search.best_params_

{'columntransformer__pipeline__knnimputer__n_neighbors': 5}

In [28]:
n_best = 5

👇 What is the performance of the optimal pipeline? Make sure you cross validate! Store your result as `float` under the variable named `cv_score`

In [29]:
cv_score = None

In [36]:
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(pipe, X,y,cv=5)
cv_score = cv_score.mean()
cv_score

0.9648501785437045

In [37]:
from nbresult import ChallengeResult
result = ChallengeResult('solution', 
                         n_best = n_best,
                         cv_score=cv_score)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/cherif/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/cherif/code/cherifbenham/data-challenges/05-ML/08-Workflow/01-Preprocessor-Tuning
plugins: anyio-3.4.0
collecting ... collected 2 items

tests/test_solution.py::TestSolution::test_n_neighbours PASSED           [ 50%]
tests/test_solution.py::TestSolution::test_score_good_enough PASSED      [100%]

============================== 2 passed in 0.46s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master


👇 Using your optimal pipeline, predict wether the following tumor is malignant or not

In [38]:
new_data = pd.read_csv("new_data.csv")
new_data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902


In [39]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff51574efa0>)])),
                ('logisticregression', LogisticRegression())])

In [40]:
pipe.predict(new_data)

array([1])

🏁 Congratulation. Don't forget to add, commit and push your notebook.